<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/cudf_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing CuDF's GPU acceleration w/ Pandas dataframes

Dataset used: UK property price data from Kaggle:

https://www.kaggle.com/datasets/willianoliveiragibin/uk-property-price-data-1995-2023-04/data

In [ ]:
%load_ext cudf.pandas

First, we'll import the data and look at it.

Note the datatype for price. We're explicitly setting this, due to the original dataset being parsed as a string.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('202304.csv',
                 names=['Transaction_unique_identifier', 'price', 'Date_of_Transfer',
                        'postcode', 'Property_Type', 'Old/New',
                        'Duration', 'PAON', 'SAON', 'Street', 'Locality',
                        'Town/City', 'District', 'County', 'PPDCategory_Type',
                        'Record_Status - monthly_file_only'],
    dtype={'price': "int64"},
)

df.tail()

,Transaction_unique_identifier,price,Date_of_Transfer,postcode,Property_Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPDCategory_Type,Record_Status - monthly_file_only
28276223,{F3B6C198-17C3-6E40-E053-6C04A8C0B3B4},490000,2023-01-13 00:00,RM1 4SR,T,N,F,12,None,HEATHER GLEN,None,ROMFORD,HAVERING,GREATER LONDON,A,A
28276224,{F3B6C198-17C9-6E40-E053-6C04A8C0B3B4},365000,2023-01-10 00:00,SS2 4UA,F,N,L,"CENTENARY PLACE, 1",FLAT 22,SOUTHCHURCH BOULEVARD,None,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,A,A
28276225,{F3B6C198-17D8-6E40-E053-6C04A8C0B3B4},730000,2023-01-05 00:00,CO13 0EY,S,N,F,HIGH VIEW,None,LITTLE CLACTON ROAD,GREAT HOLLAND,FRINTON-ON-SEA,TENDRING,ESSEX,A,A
28276226,{F3B6C198-1807-6E40-E053-6C04A8C0B3B4},379000,2023-01-06 00:00,CO15 5NA,D,N,F,12,None,DULWICH ROAD,HOLLAND-ON-SEA,CLACTON-ON-SEA,TENDRING,ESSEX,A,A
28276227,{F3B6C198-182F-6E40-E053-6C04A8C0B3B4},450000,2023-01-05 00:00,CO7 7PZ,D,N,F,21,None,INGRAMS PIECE,ARDLEIGH,COLCHESTER,TENDRING,ESSEX,A,A


In [ ]:
len(df)

28276228

A simple operation to find unique values. In this case, we're going to grab for unique Towns/Cities.

In [ ]:
# get unique values for a given column:
which_unique_col = 'Town/City'
uniques = df[which_unique_col].unique()
print(len(uniques))

1172


In [ ]:
uniques[:10]

array(['MILTON KEYNES', 'SUNDERLAND', 'COLCHESTER', 'SOLIHULL',
       'BRIERLEY HILL', 'ROTHERHAM', 'PETERBOROUGH', 'MACCLESFIELD',
       'SWANSEA', 'NORWICH'], dtype=object)

Now let's do a slightly more complicated operation. We're going to find the average price for each town/city.

In [ ]:
uniques_prices = {}
for unique in uniques:
    uniques_prices[unique] = df[df[which_unique_col] == unique]['price'].mean()

uniques_prices

{'MILTON KEYNES': 216739.41150756637,
 'SUNDERLAND': 115812.21776881721,
 'COLCHESTER': 210157.37292512273,
 'SOLIHULL': 267353.89040272235,
 'BRIERLEY HILL': 123870.11858684511,
 'ROTHERHAM': 114841.08626248685,
 'PETERBOROUGH': 163094.3706850924,
 'MACCLESFIELD': 217603.70160925775,
 'SWANSEA': 134776.42491041098,
 'NORWICH': 184368.1999814746,
 'WALSALL': 148149.57452609713,
 'CHELTENHAM': 250695.22364971685,
 'BECKENHAM': 316173.69662884175,
 'LIVERPOOL': 141620.56214913898,
 'LONDON': 483432.5411589882,
 'NORTHOLT': 198671.39874440577,
 'BLACKPOOL': 99818.77619084086,
 'CORBRIDGE': 294325.6781609195,
 'WORCESTER': 193647.59425942213,
 'SLEAFORD': 140652.94869503705,
 'BRISTOL': 219496.40882534074,
 'SWINDON': 186213.48524310335,
 'CHERTSEY': 326504.30302743614,
 'LUTON': 164208.85756279257,
 'MANCHESTER': 149823.8309340425,
 'HOVE': 258300.5027065111,
 'ASHBY-DE-LA-ZOUCH': 215109.7623155505,
 'SUTTON COLDFIELD': 241525.34700516576,
 'KETTERING': 163202.93254076378,
 'CHALFONT ST. 

Next, we'll grab data just for 2022:

In [ ]:
df_2022 = df[df['Date_of_Transfer'].str.startswith('2022')]

Grab the lower 20% and upper 20% averages for properties in 2022.

In [ ]:
recent_lower_upper = {}

for unique in uniques:

    prices = df_2022[df_2022[which_unique_col] == unique]['price']
    sorted_prices = prices.sort_values()

    # bottom 20% of the prices
    bottom_20_percent_prices = sorted_prices[:int(0.2 * len(sorted_prices))]

    # average of the bottom 20%
    average_of_bottom_20_percent = bottom_20_percent_prices.mean()

    # top 20% of the prices
    top_20_percent_prices = sorted_prices[int(0.8 * len(sorted_prices)):]
    # average of the top 20%
    average_of_top_20_percent = top_20_percent_prices.mean()

    recent_lower_upper[unique] = (average_of_bottom_20_percent, average_of_top_20_percent)


recent_lower_upper

{'MILTON KEYNES': (145209.19414893616, 1040103.4488711819),
 'SUNDERLAND': (57598.83542039356, 382399.2969588551),
 'COLCHESTER': (165101.45194805195, 743320.1543450064),
 'SOLIHULL': (169737.0992063492, 954522.1623762376),
 'BRIERLEY HILL': (84892.07627118644, 393329.35294117645),
 'ROTHERHAM': (69336.18461538461, 372788.05222734256),
 'PETERBOROUGH': (129923.33620689655, 638733.4009840098),
 'MACCLESFIELD': (125174.44363636363, 1026978.036231884),
 'SWANSEA': (89393.35723431499, 473603.2340153453),
 'NORWICH': (153349.73511450383, 668095.0953470633),
 'WALSALL': (106190.06652360516, 470153.1605995717),
 'CHELTENHAM': (163773.77908937607, 1151449.372053872),
 'BECKENHAM': (243124.5748502994, 1216312.6488095238),
 'LIVERPOOL': (75708.61082352941, 476806.06302916276),
 'LONDON': (275152.6557612647, 2773460.021555368),
 'NORTHOLT': (212971.75, 998280.1470588235),
 'BLACKPOOL': (69852.94966442953, 299120.4221105528),
 'CORBRIDGE': (159136.36363636365, 776450.0),
 'WORCESTER': (139232.3620

Then from here, you might calculate range or a ratio to determine variance for a given area and then find standouts:

In [ ]:
ratios = {}
ranges = {}

for city in recent_lower_upper:
    total_range = recent_lower_upper[city][1] - recent_lower_upper[city][0]
    ratio = recent_lower_upper[city][1] / recent_lower_upper[city][0]
    #confirm ratio is a number:
    if not np.isnan(ratio) and not np.isnan(total_range):
        #print(f"City: {city}, ratio: {ratio}, total_range: {total_range}")

        ratios[city] = ratio
        ranges[city] = total_range


# top 5 ratios:
sorted_ratios = sorted(ratios.items(), key=lambda x: x[1], reverse=True)
print(f"Top 5 ratios: {sorted_ratios[:5]}")

# top 5 ranges:
sorted_ranges = sorted(ranges.items(), key=lambda x: x[1], reverse=True)
print(f"Top 5 ranges: {sorted_ranges[:5]}")



Top 5 ratios: [('KINGS LANGLEY', 122.43365828694262), ('DARTMOUTH', 38.64076101351772), ('IVER', 29.149683333950644), ('VIRGINIA WATER', 24.78180858596006), ('BUILTH WELLS', 20.304402744444165)]
Top 5 ranges: [('IVER', 8744391.240530303), ('VIRGINIA WATER', 7295441.969047619), ('ESHER', 3777076.9559356133), ('WEMBLEY', 3735620.172220459), ('WEST DRAYTON', 3287904.3513513515)]


In [ ]:
# find the earliest date of transfer:
df['Date_of_Transfer'].max()

'2023-04-27 00:00'

In [ ]:
# sort the dataframe by price:
df = df.sort_values(by=['price'], ascending=False)

